<a href="https://colab.research.google.com/github/CE334/CE334_230586_Kulshreshth_Chikara/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np

a = 6378137.0
f = 1/298.257223563
e2 = 2*f - f*f

In [8]:
#Q1: Geodetic <-> Cartesian
def geo2ecef(lat, lon, h):
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    N = a/np.sqrt(1 - e2*np.sin(lat)**2)
    x = (N+h)*np.cos(lat)*np.cos(lon)
    y = (N+h)*np.cos(lat)*np.sin(lon)
    z = ((1-e2)*N+h)*np.sin(lat)
    return x,y,z

def ecef2geo(x, y, z):
    lon = np.deg2rad( np.rad2deg(np.arctan2(y,x)) )
    p = np.sqrt(x*x + y*y)
    lat = np.arctan2(z, p*(1-e2))
    for _ in range(10):
        N = a/np.sqrt(1 - e2*np.sin(lat)**2)
        h = p/np.cos(lat) - N
        lat = np.arctan2(z, p*(1-e2*N/(N+h)))
    return np.rad2deg(lat), np.rad2deg(lon),h


#Hometown (Noida)
lat_h = 28.5355
lon_h = 77.3910
h_h   = 206.0

xh,yh,zh = geo2ecef(lat_h, lon_h, h_h)
print("ECEF of Noida:\n", xh, yh, zh, "\n")

ECEF of Noida:
 1224162.2382811958 5472548.998721041 3028872.2506504823 



In [10]:
#Q2: Spherical <-> Cartesian
def sph2cart(phi, lam, r):
    phi,lam = np.deg2rad(phi), np.deg2rad(lam)
    x = r*np.cos(phi)*np.cos(lam)
    y = r*np.cos(phi)*np.sin(lam)
    z = r*np.sin(phi)
    return x ,y,z

def cart2sph(x,y,z):
    r = np.sqrt(x*x + y*y + z*z)
    phi = np.rad2deg(np.arcsin(z/r))
    lam = np.rad2deg(np.arctan2(y,x))
    return phi,lam, r

# test Q2 small example
xp,yp,zp = sph2cart(lat_h, lon_h, 6400000)
print("Test spherical->cart:", xp,yp,zp)
print("Test cart->spherical:", cart2sph(xp,yp,zp),"\n")


Test spherical->cart: 1227380.1989599522 5486934.712427729 3057300.3301534937
Test cart->spherical: (np.float64(28.5355), np.float64(77.391), np.float64(6400000.0)) 



In [11]:
#Q3: Geodetic -> Spherical
def geo2sph(lat, lon, h):
    x,y,z = geo2ecef(lat,lon,h)
    return cart2sph(x,y,z)

print("Noida spherical:", geo2sph(lat_h, lon_h, h_h), "\n")

Noida spherical: (np.float64(28.374289919122642), np.float64(77.391), np.float64(6373494.554778941)) 



In [13]:
#Q4:Topocentric (ENU) coords satellite over Kanpur
# Kanpur coords from lab sheet
lat_k = 26 + 26/60 + 59.7228/3600
lon_k = 80 + 19/60 + 54.7356/3600


# +800 km +local height
h_sat = 800000 + 126.63

xs,ys,zs = geo2ecef(lat_k, lon_k, h_sat)

# ENU transform relative to Noida
def ecef2enu(dx, dy, dz, lat0, lon0):
    lat0, lon0 = np.deg2rad(lat0), np.deg2rad(lon0)
    sl, cl = np.sin(lat0), np.cos(lat0)
    sm, cm = np.sin(lon0), np.cos(lon0)
    R = np.array([
        [-sm,       cm,      0],
        [-cl*cm, -cl*sm,  sl],
        [ sl*cm,  sl*sm,  cl]])
    return R @ np.array([dx,dy,dz])

dx,dy,dz = xs-xh, ys-yh, zs-zh
e,n,u = ecef2enu(dx,dy,dz, lat_h, lon_h)

print("Satellite ECEF:", xs,ys,zs)
print("Delta ECEF:", dx,dy,dz)
print("\nSatellite ENU relative to Noida (m):")
print("E =", e)
print("N =", n)
print("U =", u)

Satellite ECEF: 1079973.0888587618 6339333.117883207 3180167.2081966875
Delta ECEF: -144189.14942243393 866784.1191621665 151294.95754620526

Satellite ENU relative to Noida (m):
E = 329927.6354203175
N = -643196.6939139748
U = 521958.95676374447
